**Ways to make it better:**



*   One-hot Encoding
*   Better Model

## Read and Prepare dataset



In [0]:
# Upload and decompress file
import os

if not os.path.exists("heart-attack-prediction.csv"):
    !rm -rf sample_data
    from google.colab import files
    files.upload()
    !xz -d *.xz
    print("Uploaded files:")
    !ls
else:
    print("Already have files:")
    !ls

Already have files:
heart-attack-prediction.csv


In [0]:
# Read CSV, prepare dataframe, split x and y
import numpy as np
import pandas as pd

dataset = pd.read_csv("heart-attack-prediction.csv")

# converting all string values to nan
dataset = dataset.convert_objects(convert_numeric=True)

# drop columns with too many NaNs
dataset = dataset.drop(["ca", "thal"], axis=1)

# one-hot cp

# split x, y
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


In [0]:
# Fill in NaNs, scale
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = "NaN", strategy ="mean", axis = 0)
imputer = imputer.fit(x[:,0:13])   
x[:, 0:13] = imputer.transform(x[:, 0:13])

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x = sc_X.fit_transform(x)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
#@title Training options
metric = "f1" #@param ["f1", "f2", "precision", "recall", "accuracy"]
training_attempts = 1000 #@param {type:"slider", min:0, max:1000, step:1}


In [0]:
best_cm = [[0,0],[0,0]]

best_f1 = 0
best_f2 = 0
best_precision = 0
best_recall = 0
best_accuracy = 0
best_weights = []

metric_decision = eval("lambda f1, f2, precision, recall, accuracy: %s > best_%s"%(metric, metric))

def calculate_metrics(cm):
    tn, fp, fn, tp = cm.ravel()
    precision = tp/(tp+fp) # How many of the people we identify actualy have heart attacks?
    recall = tp/(tp+fn) # How many of the heart attacks did we predict
    accuracy = (tp+tn)/(tp+tn+fp+fn) # How accurate were our predictions
    f1 = 2*(precision*recall)/(precision+recall) # Even mix of precision and recall
    f2 = 5*(precision*recall)/(4*precision+recall) # Weighs recall more
    return f1, f2, precision, recall, accuracy


def save_best(cm, f1, f2, precision, recall, accuracy, weights):
    if metric_decision(f1, f2, precision, recall, accuracy):
        for metric in ["cm", "f1", "f2", "precision", "recall", "accuracy", "weights"]:
            exec("global best_%s; best_%s = %s"%(metric, metric, metric))



In [0]:
from sklearn.model_selection import train_test_split
x_train_cv, x_test, y_train_cv, y_test = train_test_split(x, y, test_size = 0.2)

In [0]:
from tqdm import tqdm

for i in tqdm(range(training_attempts)):
    
    x_train,x_cv,y_train,y_cv = train_test_split(x_train_cv , y_train_cv , test_size=0.1)
    
    from sklearn.linear_model import LogisticRegression
    result = LogisticRegression(random_state = 0, solver = "lbfgs")
    result.fit(x_train,y_train)

    y_predict = result.predict(x_cv)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_cv,y_predict)
    
    f1, f2, precision, recall, accuracy = calculate_metrics(cm)
    weights = np.hstack((result.intercept_[:,None], result.coef_*np.std(x, 0)))
    
    save_best(cm, f1, f2, precision, recall, accuracy, weights)

100%|██████████| 1000/1000 [00:04<00:00, 247.81it/s]


In [0]:
y_predict_test = result.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_predict_test)

f1, f2, precision, recall, accuracy = calculate_metrics(cm)


In [0]:
results = pd.DataFrame({"Column": dataset.columns, "Weight": best_weights[0]})
results.Column[len(results.Column)-1] = "Bias (y-intercept)"
results

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Column,Weight
0,age,-0.910333
1,sex,0.186322
2,cp,0.554563
3,trestbps,0.912624
4,chol,-0.153888
5,fbs,0.338960
6,restecg,0.455813
7,thalach,-0.117375
8,exang,-0.043155
9,oldpeak,0.385724


In [0]:
metrics = pd.DataFrame({"Metric": ["f1", "f2", "precision", "recall", "accuracy"], "Score": [f1, f2, precision, recall, accuracy]})
metrics

,Metric,Score
0,f1,0.722222
1,f2,0.677083
2,precision,0.812500
3,recall,0.650000
4,accuracy,0.830508


In [0]:
print("Confusion matrix:\n")
print(cm)

Confusion matrix:

[[36  3]
 [ 7 13]]
